In [22]:
import pandas as pd

melb_data=pd.read_csv('melb_data.csv', sep=',')

melb_df=melb_data.copy()
melb_df.drop(['index','Coordinates'], axis=1, inplace=True)
total_rooms=melb_df['Rooms']+melb_df['Bedroom']+melb_df['Bathroom']
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
price_square = melb_df['Price'] **2
melb_df['Date']=pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['MonthSale']=melb_df['Date'].dt.month
melb_df['AgeBuilding']=melb_df['Date'].dt.year - melb_df['YearBuilt']
melb_df.drop(['YearBuilt'],axis=1)
melb_df['WeekdaySale']=melb_df['Date'].dt.dayofweek

weekend_count=melb_df['WeekdaySale'].value_counts()[5] + melb_df['WeekdaySale'].value_counts()[6]

#display(melb_df['Address'].nunique())

from melb_data_code import get_street_type

street_types=melb_df['Address'].apply(get_street_type)

#display(street_types.value_counts(normalize=True))
# Av и Avenue, Bvd и Boulevard, Pde и Parade.)
types_long=street_types.apply(lambda x: 'Av' if x=='Avenue' else x)
types_long=types_long.apply(lambda x: 'Bvd' if x=='Boulevard' else x)
types_long=types_long.apply(lambda x: 'Pde' if x=='Parade' else x)
#display(types_long.nunique())
#display(types_long.value_counts())
street_types=types_long
popular_types=street_types.value_counts().nlargest(10).index
#display(popular_types)
melb_df['Street Type']=street_types.apply(lambda x: x if x in popular_types else 'other')
#display(melb_df['Street Type'].nunique())

from melb_data_code import get_weekend
#display(melb_df['WeekdaySale'])
melb_df['WeekdaySale']=melb_df['WeekdaySale'].apply(get_weekend)
mask_1=(melb_df['WeekdaySale']==1)
#round(melb_df[mask_1]['Price'].mean())
melb_df['SellerG'].nunique()
largest_49_sellers=melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG']=melb_df['SellerG'].apply(lambda x: x if x in largest_49_sellers else 'other')
mask_1=(melb_df['SellerG']=='Nelson')
min_price_sold_by_nelson=melb_df[mask_1]['Price'].min()
#display(min_price_sold_by_nelson)
mask_2=(melb_df['SellerG']=='other')
min_price_sold_by_other=melb_df[mask_2]['Price'].min()
#display(min_price_sold_by_other)
#display(round((min_price_sold_by_nelson/min_price_sold_by_other),1))

# melb_df.info() #2.8+ MB
melb_df['Suburb'].nunique()
# 119
popular_119_suburb= melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb']=melb_df['Suburb'].apply(lambda x: x if x in popular_119_suburb else 'other')
melb_df['Suburb'].info()
melb_df['Suburb']=melb_df['Suburb'].astype('category')
melb_df=melb_df.drop('Address',axis=1)

#Category
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes)     # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
#display(unique_counts)

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df.info())

melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
#display(melb_df['Type'])

melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
#display(new_houses_types)

melb_df.info()

<class 'pandas.core.series.Series'>
RangeIndex: 13580 entries, 0 to 13579
Series name: Suburb
Non-Null Count  Dtype 
--------------  ----- 
13580 non-null  object
dtypes: object(1)
memory usage: 106.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  float64       
 10  Bathroom         13580 non-null  float64     

In [10]:
import pandas as pd

countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
countries_df['density']=(countries_df['population']*1000000)/countries_df['square']
#display(round(countries_df['density'].mean(),2))
#display(countries_df)